### Model traning period: 2007-03 ~ 2020-03
### Model testing period: 2020-04 ~ 2024-04
### Asset pool : ["SPY","AGG","QQQ","VNQ","IWM","VIG","IEF"]
### Variable:

    #### 市場面 ["^VIX","^GSPC","^DJI","^IXIC","^RUT","^W5000FLT","GC=F","CL=F"]
    #### 技術指標
    #### 基本面
    #### 總體經濟面
    #### 利率指標面

In [13]:
import yfinance as yf
import pandas as pd
from datetime import datetime
from dateutil import rrule

In [ ]:

def get_days_intervals(start_date, end_date):
    """
    Convert a date range to a list of day intervals
    
    Parameters:
    ----------
    start_date: datetime
    end_date: datetime
    
    Returns:
    -------
    list of datetime
    """
    # Convert start_date and end_date to datetime objects
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    
    # Generate a list of the first day of each month between start and end dates
    months = list(rrule.rrule(rrule.DAILY, dtstart=start, until=end))
    month_intervals =  [date.strftime("%Y-%m-%d") for date in months if date.isoweekday()  not in [6,7]]
    
    return month_intervals

In [ ]:

class Yf_finance:
    def __init__(self, start_date, end_date, stock_list):
        self.start_date = start_date
        self.end_date = end_date
        self.stock_list = stock_list
        self.days_intervals = get_days_intervals(start_date, end_date)

    def get_trade_day(self):
        df = yf.download('^DJI','2007-03-01','2024-03-31', interval='1d')
        df = df.reset_index()
        trade_day = df['Date']

        return trade_day

    def fundamentals(self):
        """
        Get fundamentals of stocks
        
        Parameters:
        ----------
        """
        result = pd.DataFrame({'Date':self.days_intervals})
        for s in self.stock_list:
            df = yf.download(s,self.start_date,self.end_date, interval='1d')
            df = df.reset_index()
            df = df[['Date','Close']]
            result.merge(df, on='Date', how='left')

        


In [15]:
start_date = '2007-01-01'
end_date = '2024-03-31'
stock_list = ["SPY","AGG","QQQ","VNQ","IWM","VIG","IEF"]

trade_day = yf.download('VNQ','2007-03-01','2024-03-31', interval='1d')
trade_day = trade_day.reset_index()
trade_day = trade_day['Date']

[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['VNQ']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


In [17]:
### 基本面
# fund_raw = pd.DataFrame(trade_day)
fundindex_list = ["^VIX","^GSPC","^DJI","^IXIC","^RUT","^W5000FLT","GC=F","CL=F"]
for s in fundindex_list:
    df = yf.download(s,'2007-03-01','2024-03-31', interval='1d')
    df = df.reset_index()
    # df = df[['Date','Close']]
    # df = df.rename(columns = {'Close':f'{s}'})
    # fund_raw = fund_raw.merge(df, on ='Date', how = 'left')
    

[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['^VIX']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


In [ ]:
### 基本面相關
result =pd.DataFrame(trade_day)
for s in stock_list:
    df = yf.download(s,'2007-03-01','2024-03-31', interval='1d')
    df = df.reset_index()
    df = df[['Date','Close','Volume']]
    df = df.rename(columns = {'Close':f'{s}_Close', 'Volume':f'{s}_Volume'})
    result = result.merge(df, on='Date', how='left')
    result.fillna(method = 'ffill', inplace = True)
result.to_csv(f'./asset_price.csv', index = False)


In [ ]:
#利率指標
US1M = pd.read_csv('./data/archived/US1M.csv')
US1M['Date'] = pd.to_datetime(US1M['Date'])
US3M = pd.read_csv('./data/archived/US3M.csv')
US3M = US3M.rename(columns = {'Date\t':'Date'})
US3M['Date'] = pd.to_datetime(US3M['Date'])
DGS5 = pd.read_csv('./data/archived/DGS5.csv')
DGS5['Date'] = pd.to_datetime(DGS5['Date'])
DGS10 = pd.read_csv('./data/archived/DGS10.csv')
DGS10['Date'] = pd.to_datetime(DGS10['Date'])
DGS10 = DGS10[['Date','Close_DGS10']]
DFF = pd.read_csv('./data/DFF.csv')
DFF['Date'] = pd.to_datetime(DFF['Date'])

bond_rate =pd.DataFrame(trade_day)
bond_rate = bond_rate.merge(US1M, on='Date', how='left')
bond_rate = bond_rate.merge(US3M, on='Date', how='left')
bond_rate = bond_rate.merge(DGS5, on='Date', how='left')
bond_rate = bond_rate.merge(DGS10, on='Date', how='left')
bond_rate = bond_rate.merge(DFF, on='Date', how='left')

bond_rate.to_csv('./data/bond_rate.csv', index = False)

In [ ]:
#技術指標
df = pd.read_csv('./data/asset_price.csv')
df = df[['Date','SPY_Close','AGG_Close','QQQ_Close','VNQ_Close','IWM_Close','VIG_Close','IEF_Close']]

In [ ]:
#計算14日RSI
for i in df.columns[1:]:
    stock = i.replace('_Close','')
    df[f'{stock}_change'] = df[i].diff()
    df[f'{stock}_change'] = df[f'{stock}_change'].fillna(0)
    
    # 分別計算漲幅和跌幅
    df[f'{stock}_gain'] = df[f'{stock}_change'].mask(df[f'{stock}_change'] < 0, 0)
    df[f'{stock}_loss'] = -df[f'{stock}_change'].mask(df[f'{stock}_change'] > 0, 0)

    # 計算平均漲幅和平均跌幅
    df[f'{stock}_avg_gain'] = df[f'{stock}_gain'].rolling(window=14, min_periods=14).mean()
    df[f'{stock}_avg_loss'] = df[f'{stock}_loss'].rolling(window=14, min_periods=14).mean()

    # 計算RS和RSI
    df[f'{stock}_rs'] = df[f'{stock}_avg_gain'] / df[f'{stock}_avg_loss']
    df[f'{stock}_rsi'] = 100 - (100 / (1 + df[f'{stock}_rs']))





In [ ]:
#計算20日、50日、100日簡單移動平均線
from stockstats import StockDataFrame as Sdf
result = pd.read_csv('./asset_price.csv')
result['Date'] = pd.to_datetime(result['Date'])
for s in stock_list:
    df = yf.download(s,'2007-03-01','2024-03-31', interval='1d')
    df = df.reset_index()
    df = df[['Date','Close','High','Low','Volume']]
    # 將DataFrame轉換為StockDataFrame
    stock = Sdf.retype(df)
    df[f'{s}_close_20_sma'] = stock['close_20_sma']
    df[f'{s}_close_50_sma'] = stock['close_50_sma']
    df[f'{s}_close_100_sma'] = stock['close_100_sma']

    df = df.reset_index()
    df = df.rename(columns = {'date':'Date'})
    df = df[['Date',f'{s}_close_20_sma',f'{s}_close_50_sma',f'{s}_close_100_sma']]

    result = result.merge(df, on='Date', how='left')

result.to_csv('./data/asset_sma.csv', index = False)


In [ ]:
#平均回報率

result =pd.DataFrame(trade_day)

# 定义资产和时间段
periods = [20, 60, 120, 180, 240]  # 假设每个月大约有20个交易日


# 计算回报率
# returns = pd.DataFrame(index=assets, columns=[f'{p}d' for p in periods])
for asset in stock_list:
    # 下载数据
    data = yf.download(asset, '2007-03-01','2024-03-31', interval='1d')['Adj Close']
    data = data.reset_index()
    for period in periods:
        # returns.loc[asset, f'{period}d'] = data[asset].pct_change(periods=period).iloc[-1]
        data = data.rename(columns = {'Adj Close' : f'{asset}_adj_close'})
        data[f'{asset}_{period}'] = data[f'{asset}_adj_close'].pct_change(periods=period)
    data = data.fillna(0)
    column_to_sum = [f'{asset}_20', f'{asset}_60', f'{asset}_120', f'{asset}_180', f'{asset}_240']
    data[f'{asset}_sum'] = data[column_to_sum].sum(axis=1)
    data[f'{asset}_avg'] = data[f'{asset}_sum'] / 5
    
    result = result.merge(data, on='Date', how='left')
    result.to_csv('./data/asset_monetum.csv',index=False)
# # 分配分数
# scores = returns.rank(ascending=False)

# # 计算总分
# scores['Total'] = scores.sum(axis=1)


In [ ]:
#計算TIPS 的絕對動能 
data = yf.download('TIP', '2006-03-01','2024-03-31', interval='1d')['Adj Close']
data = data.reset_index()
data = data.rename(columns = {'Adj Close' : f'TIPS_adj_close'})
data[f'TIPS_240'] = data['TIPS_adj_close'].rolling(window=240).mean()
data = data.fillna(0)
data['TIPS_momentum'] = data['TIPS_adj_close'] - data['TIPS_240']
data = data[data['Date']>= '2007-03-01']
data.to_csv('./data/TIPS_signal.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# 假設你已經有一個DataFrame 'df'，包含了日期和收盤價格
# df = pd.read_csv('your_data.csv')  # 如果你的數據在CSV文件中

# 繪製收盤價格的趨勢線圖
plt.figure(figsize=(10, 5))  # 設定圖形大小
plt.plot(data['Date'], data['TIPS_momentum'], label='TIPS_momentum')  # 繪製收盤價格趨勢線

plt.axhline(0, color='red', linewidth=0.5)
# 設定圖表標題和軸標籤
plt.title('momentum Trend')
plt.xlabel('Date')
plt.ylabel('TIPS_momentum')

# 顯示圖例
plt.legend()

# 優化日期標籤顯示
plt.gcf().autofmt_xdate()

# 顯示圖表
plt.show()


In [ ]:
# 總體經濟面
CPI = pd.read_csv('./data/CPIAUCSL.csv')
CPI['DATE'] = pd.to_datetime(CPI['DATE'])
CSU = pd.read_csv('./data/CSUSHPINSA.csv')
CSU['DATE'] = pd.to_datetime(CSU['DATE'])
fred = pd.read_csv('./data/fredgraph.csv')
fred['DATE'] = pd.to_datetime(fred['DATE'])
GDP = pd.read_csv('./data/GDP.csv')
GDP['DATE'] = pd.to_datetime(GDP['DATE'])
GDPC1 = pd.read_csv('./data/GDPC1.csv')
GDPC1['DATE'] = pd.to_datetime(GDPC1['DATE'])

result =pd.DataFrame(trade_day)
result = result.merge(CPI, left_on='Date',right_on='DATE', how='left')[['Date', 'CPIAUCSL']]
result = result.merge(CSU, left_on='Date',right_on='DATE', how='left')[['Date', 'CPIAUCSL','CSUSHPINSA']]
result = result.merge(fred, left_on='Date',right_on='DATE', how='left')[['Date', 'CPIAUCSL','CSUSHPINSA','CPIAUCSL_PC1','CPILFESL_PC1']]
result = result.merge(GDP, left_on='Date',right_on='DATE', how='left')[['Date', 'CPIAUCSL','CSUSHPINSA','CPIAUCSL_PC1','CPILFESL_PC1','GDP']]
result = result.merge(GDPC1, left_on='Date',right_on='DATE', how='left')[['Date', 'CPIAUCSL','CSUSHPINSA','CPIAUCSL_PC1','CPILFESL_PC1','GDP','GDPC1']]

result.fillna(method = 'ffill', inplace = True)
# result['GDPC1'].fillna(16611.69, inplace=True)
# result['GDP'].fillna(14215.651, inplace=True)
result.to_csv('FRED_data.csv', index=False)


In [ ]:
fund = pd.read_csv('./data/fundamentals_1.csv')
aaii = pd.read_csv('./data/AAII.csv')
fund['Date'] = pd.to_datetime(fund['Date'])
aaii['Reported'] = pd.to_datetime(aaii['Reported'])

In [ ]:
fund = fund.merge(aaii, left_on = 'Date', right_on = 'Reported', how='left')
fund.drop('Reported', axis=1, inplace=True)
fund.fillna(method = 'ffill', inplace = True)
fund.to_csv('./data/fundamentals.csv', index=False)

In [ ]:
aaii

In [ ]:

fund['Bull-Bear Spread'] = fund['Bull-Bear Spread'].str.replace('%', '', regex=False).astype('float')/100